# Intelligent Incident Response Assistant

This notebook provides an AI-powered incident response workflow using:
- **Claude AI** for intelligent analysis and recommendations
- **MCP integrations** for security tool APIs (CrowdStrike, Microsoft, Proofpoint)
- **Machine Learning** for anomaly detection and threat classification
- **Automated workflows** for common incident response tasks

## Features
1. Automated incident triage and prioritization
2. AI-powered threat analysis and recommendations
3. Cross-platform security data correlation
4. Automated response actions
5. Incident timeline reconstruction
6. Report generation

In [ ]:
# Import required libraries
import os
import sys
import json
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from anthropic import Anthropic
import boto3

# Add local modules to path
sys.path.append('/home/sagemaker-user/lib')

# Import security tool integrations
from security_integrations.crowdstrike_client import CrowdStrikeClient
from security_integrations.microsoft_client import MicrosoftSecurityClient
from security_integrations.proofpoint_client import ProofpointClient
from security_integrations.ai_agent import IncidentResponseAgent

# Configure visualization
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ Libraries imported successfully")

## 1. Initialize Security Tool Clients and AI Agent

In [ ]:
# Initialize security tool clients
crowdstrike = CrowdStrikeClient()
microsoft = MicrosoftSecurityClient()
proofpoint = ProofpointClient()

# Initialize AI agent for incident response
ir_agent = IncidentResponseAgent(
    crowdstrike=crowdstrike,
    microsoft=microsoft,
    proofpoint=proofpoint
)

print("✓ Security clients and AI agent initialized")

## 2. Incident Discovery and Triage

Automatically discover and triage incidents from all security platforms

In [ ]:
# Set time window for incident discovery (last 24 hours)
start_time = (datetime.utcnow() - timedelta(hours=24)).isoformat() + 'Z'

# Discover incidents across all platforms
print("🔍 Discovering incidents across all security platforms...\n")

# Get CrowdStrike detections
cs_detections = await crowdstrike.get_detections(
    start_time=start_time,
    severity='High'
)
print(f"CrowdStrike High Severity Detections: {cs_detections['count']}")

# Get CrowdStrike incidents
cs_incidents = await crowdstrike.get_incidents(
    start_time=start_time,
    status='New'
)
print(f"CrowdStrike New Incidents: {cs_incidents['count']}")

# Get Microsoft Defender alerts
ms_alerts = await microsoft.get_defender_alerts(
    start_time=start_time,
    severity='high'
)
print(f"Microsoft Defender High Severity Alerts: {ms_alerts['count']}")

# Get Proofpoint threat events
pp_events = await proofpoint.get_siem_events(
    interval='PT24H',
    threat_status='active'
)
print(f"Proofpoint Active Threats: {pp_events['total_events']}")

# Combine all incidents for triage
all_incidents = ir_agent.aggregate_incidents(
    cs_detections=cs_detections,
    cs_incidents=cs_incidents,
    ms_alerts=ms_alerts,
    pp_events=pp_events
)

print(f"\n📊 Total incidents to triage: {len(all_incidents)}")

## 3. AI-Powered Incident Analysis

Use Claude AI to analyze incidents, identify patterns, and provide recommendations

In [ ]:
# Analyze incidents with AI
print("🤖 AI Agent analyzing incidents...\n")

analysis_results = await ir_agent.analyze_incidents(all_incidents)

# Display prioritized incidents
print("=" * 80)
print("HIGH PRIORITY INCIDENTS")
print("=" * 80)

for incident in analysis_results['high_priority']:
    print(f"\n🚨 {incident['title']}")
    print(f"   Source: {incident['source']}")
    print(f"   Severity: {incident['severity']}")
    print(f"   AI Risk Score: {incident['ai_risk_score']}/100")
    print(f"   Recommended Actions:")
    for action in incident['recommended_actions']:
        print(f"     • {action}")
    print(f"   Reasoning: {incident['ai_reasoning']}")

# Display summary statistics
print("\n" + "=" * 80)
print("ANALYSIS SUMMARY")
print("=" * 80)
print(f"High Priority: {len(analysis_results['high_priority'])}")
print(f"Medium Priority: {len(analysis_results['medium_priority'])}")
print(f"Low Priority: {len(analysis_results['low_priority'])}")
print(f"\nPotential Attack Campaigns Detected: {len(analysis_results['campaigns'])}")

## 4. Deep Dive Investigation

Investigate a specific high-priority incident in detail

In [ ]:
# Select incident for investigation (use first high-priority incident)
if analysis_results['high_priority']:
    incident_to_investigate = analysis_results['high_priority'][0]
    
    print(f"🔬 Deep dive investigation: {incident_to_investigate['title']}\n")
    
    # Perform automated investigation
    investigation = await ir_agent.investigate_incident(
        incident=incident_to_investigate,
        include_timeline=True,
        include_related_events=True,
        include_threat_intel=True
    )
    
    # Display timeline
    print("📅 INCIDENT TIMELINE")
    print("=" * 80)
    for event in investigation['timeline']:
        print(f"{event['timestamp']} - {event['description']}")
        print(f"  Source: {event['source']}")
        if event.get('indicators'):
            print(f"  Indicators: {', '.join(event['indicators'])}")
        print()
    
    # Display related events
    print("\n🔗 RELATED EVENTS")
    print("=" * 80)
    for event in investigation['related_events']:
        print(f"• {event['description']} ({event['correlation_confidence']}% confidence)")
    
    # Display threat intelligence
    print("\n🛡️ THREAT INTELLIGENCE")
    print("=" * 80)
    print(f"Known Threat Actor: {investigation['threat_intel'].get('actor', 'Unknown')}")
    print(f"TTPs: {', '.join(investigation['threat_intel'].get('ttps', []))}")
    print(f"Malware Family: {investigation['threat_intel'].get('malware_family', 'Unknown')}")
    
    # AI-generated investigation summary
    print("\n🤖 AI INVESTIGATION SUMMARY")
    print("=" * 80)
    print(investigation['ai_summary'])
else:
    print("No high-priority incidents to investigate")

## 5. Affected Assets and User Analysis

In [ ]:
# Get affected assets
if investigation:
    affected_assets = investigation.get('affected_assets', {})
    
    # Analyze hosts
    print("💻 AFFECTED HOSTS")
    print("=" * 80)
    for host in affected_assets.get('hosts', []):
        host_details = await crowdstrike.get_host_info(host)
        print(f"\nHostname: {host}")
        if 'error' not in host_details:
            h = host_details['host']
            print(f"  OS: {h.get('os_version')}")
            print(f"  IP Address: {h.get('local_ip')}")
            print(f"  Last Seen: {h.get('last_seen')}")
            print(f"  Risk Score: {h.get('reduced_functionality_mode', 'Normal')}")
    
    # Analyze users
    print("\n👤 AFFECTED USERS")
    print("=" * 80)
    for user in affected_assets.get('users', []):
        # Get user info from Entra ID
        user_info = await microsoft.get_entra_users(
            filter_query=f"userPrincipalName eq '{user}'"
        )
        
        if user_info.get('users'):
            u = user_info['users'][0]
            print(f"\nUser: {u['user_principal_name']}")
            print(f"  Display Name: {u.get('display_name')}")
            print(f"  Department: {u.get('department')}")
            print(f"  Job Title: {u.get('job_title')}")
            
            # Check if user is in risky users list
            risky_users = await microsoft.get_risky_users()
            is_risky = any(ru['user_principal_name'] == user for ru in risky_users.get('risky_users', []))
            if is_risky:
                print(f"  ⚠️ WARNING: User is flagged as risky in Entra ID Protection")
            
            # Get recent sign-in activity
            sign_ins = await microsoft.get_sign_in_logs(
                user_principal_name=user,
                limit=5
            )
            print(f"  Recent Sign-ins:")
            for signin in sign_ins.get('sign_ins', [])[:3]:
                print(f"    • {signin['created_datetime']} from {signin['ip_address']} ({signin['location']})")

## 6. Automated Response Actions

Execute automated containment and remediation actions

In [ ]:
# Define response playbook
response_playbook = {
    'containment': [
        'isolate_affected_hosts',
        'disable_compromised_accounts',
        'block_malicious_ips'
    ],
    'eradication': [
        'remove_malware',
        'reset_credentials',
        'patch_vulnerabilities'
    ],
    'recovery': [
        'restore_from_backup',
        're_enable_accounts',
        'remove_host_isolation'
    ]
}

# Execute containment phase (with approval)
print("🚨 RECOMMENDED CONTAINMENT ACTIONS")
print("=" * 80)

recommended_actions = ir_agent.generate_response_plan(
    incident=incident_to_investigate,
    investigation=investigation,
    playbook=response_playbook
)

for i, action in enumerate(recommended_actions['containment'], 1):
    print(f"{i}. {action['description']}")
    print(f"   Impact: {action['impact']}")
    print(f"   Risk Level: {action['risk_level']}")
    print(f"   Automated: {'Yes' if action['can_automate'] else 'No (requires approval)'}")
    print()

# Execute approved automated actions
execute_auto = input("\nExecute automated containment actions? (yes/no): ")

if execute_auto.lower() == 'yes':
    print("\n⚙️ Executing containment actions...\n")
    
    results = await ir_agent.execute_response_actions(
        actions=recommended_actions['containment'],
        auto_approve_safe_actions=True
    )
    
    for result in results:
        status_icon = "✓" if result['success'] else "✗"
        print(f"{status_icon} {result['action']}: {result['message']}")
else:
    print("Containment actions not executed (user declined)")

## 7. Incident Report Generation

Generate comprehensive incident report with AI-powered analysis

In [ ]:
# Generate incident report
print("📝 Generating incident report...\n")

report = ir_agent.generate_incident_report(
    incident=incident_to_investigate,
    investigation=investigation,
    response_actions=results if execute_auto.lower() == 'yes' else [],
    include_executive_summary=True,
    include_technical_details=True,
    include_recommendations=True
)

# Display report
print("=" * 80)
print("INCIDENT RESPONSE REPORT")
print("=" * 80)
print(f"\nGenerated: {datetime.utcnow().isoformat()}")
print(f"Incident ID: {report['incident_id']}")
print(f"\n{report['content']}")

# Save report to S3
s3_client = boto3.client('s3')
bucket_name = os.getenv('SAGEMAKER_NOTEBOOKS_BUCKET', 'sagemaker-infosec-notebooks')
report_key = f"incident-reports/{report['incident_id']}.md"

s3_client.put_object(
    Bucket=bucket_name,
    Key=report_key,
    Body=report['content'],
    ContentType='text/markdown'
)

print(f"\n✓ Report saved to s3://{bucket_name}/{report_key}")

## 8. Visualization and Metrics

In [ ]:
# Create visualizations
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Incidents by severity
severity_counts = pd.Series({
    'High': len(analysis_results['high_priority']),
    'Medium': len(analysis_results['medium_priority']),
    'Low': len(analysis_results['low_priority'])
})
severity_counts.plot(kind='bar', ax=axes[0, 0], color=['red', 'orange', 'yellow'])
axes[0, 0].set_title('Incidents by Severity')
axes[0, 0].set_ylabel('Count')

# Incidents by source
source_data = pd.DataFrame(all_incidents)
source_counts = source_data['source'].value_counts()
source_counts.plot(kind='pie', ax=axes[0, 1], autopct='%1.1f%%')
axes[0, 1].set_title('Incidents by Source')

# Incident timeline
if investigation and 'timeline' in investigation:
    timeline_df = pd.DataFrame(investigation['timeline'])
    timeline_df['timestamp'] = pd.to_datetime(timeline_df['timestamp'])
    timeline_df.set_index('timestamp', inplace=True)
    timeline_df.resample('H').size().plot(ax=axes[1, 0])
    axes[1, 0].set_title('Event Timeline (Hourly)')
    axes[1, 0].set_ylabel('Events')

# AI risk scores distribution
if analysis_results['high_priority']:
    risk_scores = [inc['ai_risk_score'] for inc in analysis_results['high_priority']]
    axes[1, 1].hist(risk_scores, bins=10, color='red', alpha=0.7)
    axes[1, 1].set_title('AI Risk Score Distribution (High Priority)')
    axes[1, 1].set_xlabel('Risk Score')
    axes[1, 1].set_ylabel('Count')

plt.tight_layout()
plt.savefig('/tmp/incident_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Visualizations generated")

## 9. Continuous Monitoring

Set up continuous monitoring for ongoing threat detection

In [ ]:
# Configure continuous monitoring
monitoring_config = {
    'check_interval': 300,  # 5 minutes
    'alert_threshold': 7,  # AI risk score threshold
    'sources': ['crowdstrike', 'microsoft', 'proofpoint'],
    'notification_channel': 'sns',
    'auto_triage': True
}

print("Setting up continuous monitoring...")
print(f"Check interval: {monitoring_config['check_interval']} seconds")
print(f"Alert threshold: {monitoring_config['alert_threshold']}/10")
print(f"Monitoring sources: {', '.join(monitoring_config['sources'])}")

# Note: In production, this would be deployed as a Lambda function or
# SageMaker Processing job for continuous execution

print("\n✓ Monitoring configuration saved")
print("To deploy continuous monitoring, run: python deploy_monitoring.py")